<a href="https://colab.research.google.com/github/VU-CSP/quantbio-assignments/blob/main/ThunorColab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!python --version

In [ ]:
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

if IN_COLAB:
    !pip install thunor

In [ ]:
import os, sys
import thunor

## Load a file
First, specify a file to load. Here, we use an example dataset from the thunor package itself.

In [ ]:
if not os.path.exists('./hts007.h5'):
    !wget https://github.com/alubbock/thunor/raw/master/thunor/testdata/hts007.h5

hts007_file = 'hts007.h5'

Load the file using `read_hdf` (for HDF5 files), `read_vanderbilt_hts` (for CSV files), or another appropriate reader.

In [ ]:
from thunor.io import read_hdf
hts007 = read_hdf(hts007_file)

Show a summary of the data.

In [ ]:
hts007

We'll just use a subset of the drugs, to make the plots manageable.

In [ ]:
hts007r = hts007.filter(drugs=['cediranib', 'everolimus', 'paclitaxel'])
hts007r.drugs

In [ ]:
hts007r.cell_lines

## Calculate DIP rates and parameters

These two operations can be done in two lines of code (plus imports). Note that you may see `RuntimeWarning` messages, which indicates that some dose response curves were not able to be fitted. This can happen if the cells do not stop proliferating in response to drug, the response is not closely approximated by a log-logistic curve, or the data are very noisy.

In [ ]:
from thunor.dip import dip_rates
from thunor.curve_fit import fit_params

ctrl_dip_data, expt_dip_data = dip_rates(hts007r)
fp = fit_params(ctrl_dip_data, expt_dip_data)

## Setting up plots

Each of the `plot_X` functions returns a plotly `Figure` object which can be visualised in a number of ways. Here, we use the offline `iplot` function, which generates a plot for use with Jupyter notebook. We could also generate plots using the `plot` function in standalone HTML files. See the [plotly documentation](https://plot.ly/python/offline/) for more information on the latter approach. 

In [ ]:
from thunor.plots import plot_drc, plot_drc_params, plot_time_course, plot_ctrl_dip_by_plate, plot_plate_map

## Plot Types

### Plot DIP rate curves

In [ ]:
plot_drc(fp)

### Plot DIP rate fit parameters


In [ ]:
plot_drc_params(fp, 'auc')

### Filtering fit params

The `fp` object is a pandas data frame, so we can filter it before plotting. Some examples:

In [ ]:
fit_params_bt20_pac = fp[fp.index.isin(['BT20'], level='cell_line') & \
                         fp.index.isin(['paclitaxel'], level='drug')]

if not IN_COLAB:
    bt20_pac_drc = plot_drc(fit_params_bt20_pac)
    bt20_pac_drc.write_html('drc.html', auto_open=True)

In [ ]:
plot_drc(fit_params_bt20_pac)

### Plot time course

Time course plot for paclitaxel on BT20 cells:

In [ ]:
if not IN_COLAB:
    tc = plot_time_course(hts007.filter(drugs=['paclitaxel'], cell_lines=['BT20']))
    tc.write_html('time_course.html', auto_open=True)

In [ ]:
plot_time_course(hts007.filter(drugs=['paclitaxel'], cell_lines=['BT20']))

### Quality control check: plot DIP rate ranges by cell line and plate (box plot)

In [ ]:
plot_ctrl_dip_by_plate(ctrl_dip_data)